# Settori economici e occupazione in Europa

Semplicissime analisi su: 
- valore aggiunto creato dai vari settori economici;
- impatto in termini occupazionali;

Nessuna pretesa di esaustività nè raffinatezza.

Fonte: Eurostat (sito consultato il 01/05/2020), dati scaricati dal portale della camera di commercio 
http://www.mc.camcom.it/uploaded/Allegati/Promozione/Charts/Open-Data-Tree.htm?theme=2

In [1]:
import os
import pandas as pd
import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
import sys

In [10]:
import qgrid

In [2]:
from pivottablejs import pivot_ui

In [3]:
def remove_uninformative_columns(df):
    to_return = df.copy()
    to_drop = []
    for c in df.columns:
        n = df[c].nunique()
        if n==1:
            to_drop.append(c)
            print(f'Column {c} has only one value. It wil be dropped.')
        elif df[c].count()==0:
            to_drop.append(c)
            print(f'Column {c} has only nan value. It wil be dropped.')
    return to_return.drop(to_drop,axis=1)

## Valore aggiunto per  settore economico

In [52]:
settori_economici_lu = pd.read_excel('../data/NACE_rev2_level_2.xlsx')
settori_economici_lu

,Level,Code,Description
0,1,A,"AGRICULTURE, FORESTRY AND FISHING"
1,1,B,MINING AND QUARRYING
2,1,C,MANUFACTURING
3,1,D,"ELECTRICITY, GAS, STEAM AND AIR CONDITIONING S..."
4,1,E,"WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND R..."
5,1,F,CONSTRUCTION
6,1,G,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...
7,1,H,TRANSPORTATION AND STORAGE
8,1,I,ACCOMMODATION AND FOOD SERVICE ACTIVITIES
9,1,J,INFORMATION AND COMMUNICATION


In [22]:
code  = settori_economici_lu.Code.unique()
descr = settori_economici_lu.Description.unique()

In [24]:
composed_codes = [a+'-'+b for a in code for b in code if a!=b]
composed_descr = [a+'-'+b for a in descr for b in descr if a!=b]

In [34]:
settori_economici_lu_map = dict(zip(settori_economici_lu.Code.values,settori_economici_lu.Description.values))
settori_economici_lu_map['TOTAL'] = 'TOTAL'

for k,v in zip(composed_codes, composed_descr):
    settori_economici_lu_map[k] = v

In [55]:
#correction for wrong code in table
settori_economici_lu_map['M_N'] = 'PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES-ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES'

In [56]:
valore_aggiunto_df = pd.read_excel('../data/eurostat/valore_aggiunto_eu.xlsx')
valore_aggiunto_df = remove_uninformative_columns(valore_aggiunto_df)
valore_aggiunto_df.head()

,na_item,nace_r2,unit,geo,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4
0,B1G,TOTAL,MIO_EUR_NSA,AL,2695.0,2491.8,2646.6,2482.2,3009.3,2804.8,2932.9,2717.2,3202.5,3015.5,2974.4
1,B1G,TOTAL,MIO_EUR_NSA,AT,81219.0,83288.0,86604.8,83389.1,84499.9,86103.3,90666.4,86653.2,87731.7,89124.8,92868.4
2,B1G,TOTAL,MIO_EUR_NSA,BE,100572.0,96325.0,104633.0,99005.0,103280.0,99173.0,108223.0,102217.0,106628.0,102310.0,110819.0
3,B1G,TOTAL,MIO_EUR_NSA,BG,10988.7,12368.9,12566.0,9883.8,11731.0,13541.4,13477.8,10839.6,13000.5,13950.0,14499.7
4,B1G,TOTAL,MIO_EUR_NSA,CH,149457.0,143919.3,141813.2,140712.8,143715.6,146795.9,148257.6,147230.1,151278.4,155610.6,156188.0


Usiamo i valori "seasonally and calendarly adjusted" (SCA)

In [58]:
valore_aggiunto_df = valore_aggiunto_df.loc[valore_aggiunto_df.unit=='MIO_EUR_SCA']

In [59]:
valore_aggiunto_df['nace_r2'].unique()

array(['TOTAL', 'A', 'B-E', 'C', 'F', 'G-I', 'J', 'K', 'L', 'M_N', 'O-Q',
       'R-U'], dtype=object)

In [60]:
valore_aggiunto_df['sector'] = valore_aggiunto_df['nace_r2'].map(settori_economici_lu_map)
valore_aggiunto_df.head()

,nace_r2,unit,geo,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,sector
39,TOTAL,MIO_EUR_SCA,AT,82339.0,83302.5,84007.1,85156.8,85688.2,86438.0,87616.7,88552.7,88865.2,89275.8,89645.3,TOTAL
40,TOTAL,MIO_EUR_SCA,BE,99216.0,99543.0,100712.0,101176.0,101874.0,102550.0,104079.0,104439.0,105217.0,105800.0,106518.0,TOTAL
41,TOTAL,MIO_EUR_SCA,BG,11210.2,11341.8,11582.9,11717.5,11911.1,12462.5,12461.4,12808.8,13104.8,12948.4,13423.9,TOTAL
42,TOTAL,MIO_EUR_SCA,CH,148938.4,143668.0,141299.0,142490.9,142789.8,146543.2,147817.5,149069.3,150477.2,155288.2,155663.6,TOTAL
43,TOTAL,MIO_EUR_SCA,CY,4328.1,4354.4,4399.2,4486.3,4548.3,4593.5,4632.8,4718.6,4754.4,4771.3,4827.4,TOTAL


In [61]:
valore_aggiunto_melt_df = valore_aggiunto_df.melt(id_vars=['geo','sector','nace_r2','unit']).rename(columns={'variable':'year_quarter'})
valore_aggiunto_melt_df['year'] = valore_aggiunto_melt_df['year_quarter'].apply(lambda x:x[:4])
valore_aggiunto_melt_df['quarter'] = valore_aggiunto_melt_df['year_quarter'].apply(lambda x:x[-3:])
valore_aggiunto_melt_df.head()

,geo,sector,nace_r2,unit,year_quarter,value,year,quarter
0,AT,TOTAL,TOTAL,MIO_EUR_SCA,2017Q2,82339.0,2017,Q2
1,BE,TOTAL,TOTAL,MIO_EUR_SCA,2017Q2,99216.0,2017,Q2
2,BG,TOTAL,TOTAL,MIO_EUR_SCA,2017Q2,11210.2,2017,Q2
3,CH,TOTAL,TOTAL,MIO_EUR_SCA,2017Q2,148938.4,2017,Q2
4,CY,TOTAL,TOTAL,MIO_EUR_SCA,2017Q2,4328.1,2017,Q2


In [64]:
valore_aggiunto_melt_df.loc[(valore_aggiunto_melt_df.geo=='IT')&(valore_aggiunto_melt_df.year=='2019')].head()

,geo,sector,nace_r2,unit,year_quarter,value,year,quarter
2876,IT,TOTAL,TOTAL,MIO_EUR_SCA,2019Q1,399113.5,2019,Q1
2910,IT,"AGRICULTURE, FORESTRY AND FISHING",A,MIO_EUR_SCA,2019Q1,8402.2,2019,Q1
2944,IT,"MINING AND QUARRYING-WATER SUPPLY; SEWERAGE, W...",B-E,MIO_EUR_SCA,2019Q1,78599.9,2019,Q1
2978,IT,MANUFACTURING,C,MIO_EUR_SCA,2019Q1,66694.5,2019,Q1
3012,IT,CONSTRUCTION,F,MIO_EUR_SCA,2019Q1,17580.7,2019,Q1


In [74]:
pivot_ui(valore_aggiunto_melt_df, outfile_path='valore_aggiunto.html')

In [48]:
pivot_ui(valore_aggiunto_melt_df)

## Occupazione per settore economico

In [66]:
occupati_df = pd.read_excel('../data/eurostat/occupati_in_migliaia.xlsx')
occupati_df = remove_uninformative_columns(occupati_df)
occupati_df['sector'] = occupati_df['nace_r2'].map(settori_economici_lu_map)
occupati_df.head()

Column unit has only one value. It wil be dropped.
Column s_adj has only one value. It wil be dropped.
Column na_item has only one value. It wil be dropped.


,nace_r2,geo,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,sector
0,A,AT,163.60,165.79,168.76,160.52,158.79,153.55,156.59,154.75,148.31,148.48,150.98,"AGRICULTURE, FORESTRY AND FISHING"
1,A,BE,59.30,58.30,58.10,58.50,58.60,59.00,58.80,59.20,59.00,59.30,59.40,"AGRICULTURE, FORESTRY AND FISHING"
2,A,BG,580.01,585.78,599.82,569.28,560.04,561.12,557.95,551.63,543.00,532.99,561.92,"AGRICULTURE, FORESTRY AND FISHING"
3,A,CY,15.44,15.14,14.85,15.04,14.64,14.80,14.89,14.91,15.04,15.08,15.11,"AGRICULTURE, FORESTRY AND FISHING"
4,A,CZ,162.31,162.28,161.31,160.99,160.38,160.45,161.75,162.07,158.69,156.78,156.65,"AGRICULTURE, FORESTRY AND FISHING"


In [68]:
occupati_df = occupati_df.melt(id_vars=['geo','sector','nace_r2']).rename(columns={'variable':'year_quarter'})
occupati_df['year'] = occupati_df['year_quarter'].apply(lambda x:x[:4])
occupati_df['quarter'] = occupati_df['year_quarter'].apply(lambda x:x[-3:])
occupati_df.head()

,geo,sector,nace_r2,year_quarter,value,year,quarter
0,AT,"AGRICULTURE, FORESTRY AND FISHING",A,2017Q2,163.60,2017,7Q2
1,BE,"AGRICULTURE, FORESTRY AND FISHING",A,2017Q2,59.30,2017,7Q2
2,BG,"AGRICULTURE, FORESTRY AND FISHING",A,2017Q2,580.01,2017,7Q2
3,CY,"AGRICULTURE, FORESTRY AND FISHING",A,2017Q2,15.44,2017,7Q2
4,CZ,"AGRICULTURE, FORESTRY AND FISHING",A,2017Q2,162.31,2017,7Q2


In [71]:
occupati_df.query('sector=="TOTAL" and year=="2019" and geo=="IT"')

,geo,sector,nace_r2,year_quarter,value,year,quarter
3162,IT,TOTAL,TOTAL,2019Q1,25418.6,2019,9Q1
3559,IT,TOTAL,TOTAL,2019Q2,25506.2,2019,9Q2
3956,IT,TOTAL,TOTAL,2019Q3,25544.3,2019,9Q3
4353,IT,TOTAL,TOTAL,2019Q4,25529.8,2019,9Q4


In [73]:
pivot_ui(occupati_df, outfile_path='occupazione.html')